In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/reviews-from-yandex/all_reviews.csv')

In [43]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
model.cuda()
all_emb = []

def embed_bert_cls(texts, model, tokenizer):
    # Токенизация всех текстов
    t = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    
    # Перемещение токенов на устройство модели (GPU или CPU)
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    
    # Извлечение CLS-эмбеддингов и нормализация
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    
    # Возвращаем эмбеддинги в виде numpy массива
    return embeddings.cpu().numpy()

# Общее количество предложений
num_reviews = len(df)

# Определяем размер батча
batch_size = num_reviews // 50 + (num_reviews % 50 > 0)  # Добавляем 1 к размеру батча, если есть остаток

for i in range(0, num_reviews, batch_size):
    # Пример использования функции с массивом предложений
    texts = df['review'].tolist()[i:i + batch_size]
    embeddings = embed_bert_cls(texts, model, tokenizer)

    # Печать формы эмбеддингов
    print(embeddings.shape)  # Должно быть (количество предложений в текущем батче, размерность эмбеддинга)
    all_emb.append(embeddings)

# Объединение всех эмбеддингов в один массив
all_emb = np.vstack(all_emb)

# Проверка формы окончательного массива эмбеддингов
print("Shape of all embeddings:", all_emb.shape)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(549, 312)
(500, 312)
Shape of all embeddings: (27401, 312)


In [18]:
len(all_emb)

50

In [45]:
all_emb = np.array(all_emb)
reshaped_data = all_emb.reshape(-1, 312)

In [46]:
reshaped_data.shape

(27401, 312)

In [47]:
new_df = pd.DataFrame(reshaped_data)

In [49]:
new_df

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,-0.027233,0.014551,-0.002751,-0.034192,-0.025258,-0.012406,0.058311,-0.002959,-0.004722,-0.018118,...,0.008827,0.145099,0.022913,-0.044879,0.026521,-0.035215,-0.099975,0.069299,0.126810,-0.056184
1,0.050525,0.064574,0.010602,-0.019150,0.030251,-0.022179,-0.059800,-0.028628,0.010351,0.043221,...,-0.034850,0.028944,-0.043437,0.026311,-0.049665,0.017725,-0.033748,-0.006644,0.057289,-0.095979
2,0.064398,0.064146,0.005321,0.029553,-0.022649,-0.009765,-0.021214,-0.095713,0.020693,0.049986,...,-0.015366,-0.069246,-0.003685,-0.000589,-0.030098,0.010340,-0.007372,-0.017749,0.001158,-0.047022
3,0.077163,0.066069,-0.024288,0.045249,-0.009950,-0.014215,-0.018353,-0.080334,-0.006697,0.043770,...,0.042840,-0.077443,0.008514,0.043360,-0.013460,0.014392,0.021121,-0.033954,-0.013736,-0.035103
4,0.051964,0.061968,0.013401,0.033081,-0.010398,0.014215,-0.030642,-0.023830,0.016215,0.054892,...,-0.006751,-0.003720,-0.016540,0.057570,-0.039241,0.002311,-0.048050,-0.033346,-0.024859,-0.054647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27396,0.047762,0.003337,-0.019375,-0.012891,0.063378,0.035257,0.030751,-0.084038,-0.013253,0.002279,...,0.006962,0.056569,-0.021988,0.027662,-0.024514,-0.039852,-0.041470,-0.015968,0.040738,-0.000268
27397,-0.020383,-0.003175,0.016231,-0.046817,-0.027029,0.060322,0.074528,0.074000,-0.005480,0.000737,...,0.035266,0.052416,-0.062015,-0.008782,0.004061,0.022170,-0.007891,0.068274,0.114959,-0.092255
27398,0.034853,0.024780,-0.036624,0.005077,-0.012637,0.033306,-0.053556,-0.101403,0.041082,0.013418,...,-0.032208,0.072943,-0.041472,-0.034876,-0.008240,-0.030891,-0.000090,-0.027926,0.016383,-0.052698
27399,0.053212,0.065587,0.007822,-0.019628,-0.022003,0.062872,-0.023308,-0.083250,0.005514,0.003008,...,-0.033523,-0.007374,-0.010118,0.058350,0.007866,0.008673,0.017341,0.020482,0.065723,-0.084940


In [50]:
new_df.to_csv('df_embs.csv',index=False)